# Урок 5. Задания

### Описание данных
Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

InvoiceNo – номер транзакции  

StockCode – код товара

Description – описание товара

Quantity – количество единиц товара, добавленных в заказ

InvoiceDate – дата транзакции 

UnitPrice – цена за единицу товара

CustomerID – id клиента

Country – страна, где проживает клиент



In [58]:
import pandas as pd

### Задание 1
Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.


In [59]:
# Импортируем датафрейм 
retail = pd.read_csv('/Users/ola/Downloads/[SW.BAND] 5/[SW.BAND] Задания/data.csv.zip',
                     encoding = 'ISO-8859-1',
                     compression = 'zip')

In [60]:
# Посмотрим на вид датафрейма
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [61]:
# Запишем в переменную retail_columns имена колонок датафрейма в виде списка
retail_columns = retail.columns
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

### Задание 2
Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [62]:
# Вычисляем количество повторяющихся строк
len(retail) - len(retail.drop_duplicates())

5268

In [63]:
# Количество строк и колонок ДО удаления дубликатов
retail.shape

(541909, 8)

In [64]:
# Удаление дубликатов
retail = retail.drop_duplicates()

In [65]:
# Количество строк и колонок ПОСЛЕ удаления дубликатов
retail.shape

(536641, 8)

### Задание 3
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?


In [66]:
retail.InvoiceNo.str.startswith('C').sum()

9251

### Задание 4
Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [68]:
# Фильтруем данные и перезаписываем в исходный датафрейм
retail = retail.query('Quantity > 0')

In [70]:
# Находим число оставшихся строк
retail.shape[0]

526054

### Задание 5
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany).

Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.

Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [117]:
# Посчитаем число заказов для пользователей из Германии 

cust_from_germany = retail.query('Country == "Germany"')\
                            .groupby('CustomerID', as_index = False)\
                            .agg({'InvoiceNo': 'nunique'})\
                            .rename(columns = {'InvoiceNo': 'TransCount'})
cust_from_germany

,CustomerID,TransCount
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7
...,...,...
89,13814.0,2
90,13815.0,4
91,13816.0,2
92,13817.0,2


In [118]:
# Посчитаем процентиль 80 и сохраним значение в переменную
# (80% клиентов будет иметь количество транзакций меньше этой цифры)

percentile_80 = cust_from_germany.TransCount.quantile(0.8)
percentile_80

7.0

In [122]:
# Отберем только тех клиентов, у которых количество заказов больше 80% процентиля
# Оставим только колонку CustomerID
# Запишем в переменную germany_top

germany_top = cust_from_germany.query('TransCount > @percentile_80').CustomerID
germany_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

### Задание 6
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [138]:
top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


### Задание 7
Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [144]:
top_retail_germany.query('StockCode != "POST"')\
                    .groupby('StockCode', as_index = False)\
                    .agg({'Quantity': 'count'})\
                    .sort_values('Quantity', ascending = False)

,StockCode,Quantity
409,22326,52
411,22328,38
453,22423,34
45,20719,30
709,22961,28
...,...,...
350,22168,1
769,23104,1
768,23102,1
354,22174,1


Продукт с кодом товара 22326 добавляли в корзину чаще всего

### Задание 8
Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [146]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [147]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


### Задание 9
Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [165]:
# Посчитаем финальную сумму 

inno_with_totrev = retail.groupby('InvoiceNo', as_index = False)\
                            .agg({'Revenue': 'sum'})\
                            .rename(columns = {'Revenue': 'TotalRevenue'})\
                            .sort_values('TotalRevenue', ascending = False)\
                            .head()   

inno_with_totrev

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00


In [163]:
inno_with_totrev.InvoiceNo.str.cat(sep = ', ')

'581483, 541431, 574941, 576365, 556444'